In [1]:
import tensorflow as tf
import numpy as np
import os

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [2]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 15,
    horizontal_flip= True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(x_train)

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
mean = np.mean(x_train)
std = np.std(x_train)
x_test = (x_test - mean) / std
x_train = (x_train - mean) / std
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [3]:
def resnet_block32(input_layer, filter, down_sample):
  x = input_layer
  for i in range(5):
    identity_layer = x
    x = tf.keras.layers.Conv2D(filters = filter, activation='relu', kernel_size = (3, 3), strides= (1, 1), padding='SAME', kernel_regularizer=tf.keras.regularizers.l2(0.0003), kernel_initializer=tf.keras.initializers.he_normal())(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Conv2D(filters = filter, activation='relu', kernel_size = (3, 3), strides= (1, 1), padding='SAME', kernel_regularizer=tf.keras.regularizers.l2(0.0003), kernel_initializer=tf.keras.initializers.he_normal())(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    
    identity_channel = identity_layer.shape.as_list()[-1]    
    print(identity_channel)
    if identity_channel != filter:
      identity_layer = tf.keras.layers.Conv2D(filter, kernel_size=(1, 1), strides=(1, 1), padding= 'SAME')(identity_layer) 
    x = tf.keras.layers.Add()([x, identity_layer])
  if down_sample == 0:
    x = tf.keras.layers.Conv2D(filter, kernel_size = (1, 1), strides=(2, 2), padding='same')(x)
  return x

In [4]:
x_input = tf.keras.layers.Input(shape = (32,32,3))
x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (3, 3), strides= (1, 1), padding = 'SAME',kernel_regularizer=tf.keras.regularizers.l2(0.0003), kernel_initializer=tf.keras.initializers.he_normal())(x_input)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Activation('relu')(x)

In [5]:
filters = 16
down_sample = [0, 0, 1]
for i in range(3):
  x = resnet_block32(x, filters, down_sample[i])
  filters = filters * 2
x = tf.keras.layers.AveragePooling2D(padding = 'SAME')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(units = 10, activation = 'softmax', kernel_initializer=tf.keras.initializers.he_normal())(x)

model = tf.keras.Model(inputs=x_input, outputs = x)
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9),
    metrics=['accuracy'],
)
def lr_schedule(epoch):
  if epoch > 120 :
    return 0.0001
  if epoch > 90 :
    return 0.001
  if epoch > 70 :
    return 0.01
  return 0.03
  
reduced_lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

16
16
16
16
16
16
32
32
32
32
32
64
64
64
64


C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 dropout (Dropout)              (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

In [7]:
model.fit(datagen.flow(x_train, y_train, batch_size=128), callbacks=[reduced_lr],
                       steps_per_epoch = len(x_train) // 128, epochs = 200, validation_data=(x_test, y_test), verbose = 1)

Epoch 1/200
390/390 [==============================] - 28s 59ms/step - loss: 3.5224 - accuracy: 0.2730 - val_loss: 2.6580 - val_accuracy: 0.4014 - lr: 0.0300
Epoch 2/200
390/390 [==============================] - 21s 53ms/step - loss: 2.5570 - accuracy: 0.4113 - val_loss: 2.2946 - val_accuracy: 0.4852 - lr: 0.0300
Epoch 3/200
390/390 [==============================] - 21s 53ms/step - loss: 2.2807 - accuracy: 0.4706 - val_loss: 2.0270 - val_accuracy: 0.5367 - lr: 0.0300
Epoch 4/200
390/390 [==============================] - 21s 53ms/step - loss: 2.0726 - accuracy: 0.5140 - val_loss: 1.7793 - val_accuracy: 0.6009 - lr: 0.0300
Epoch 5/200
390/390 [==============================] - 21s 53ms/step - loss: 1.8899 - accuracy: 0.5499 - val_loss: 1.7402 - val_accuracy: 0.5986 - lr: 0.0300
Epoch 6/200
390/390 [==============================] - 21s 53ms/step - loss: 1.7411 - accuracy: 0.5811 - val_loss: 1.5601 - val_accuracy: 0.6484 - lr: 0.0300
Epoch 7/200
390/390 [==============================]